In [83]:
import os
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import numpy as np
# Import relevant python packages
from gpet_oct import utils, gpet
import importlib
%matplotlib inline

In [84]:
importlib.reload(gpet)
importlib.reload(gpet_utils)

<module 'gp_edge_tracing.gpet_utils' from '/Users/uzaykaradag/Documents/School/Dissertation/gpr-oct-segmentation/gp_edge_tracing/gpet_utils.py'>

### Loading and splitting the dataset
Here the dataset is loaded as scans and masks and split into an **80-20 train-test** split. This results in **4192 train** samples from and **1048 test** samples.

In [85]:
scan_dir = 'data/scans'
mask_dir = 'data/masks'

# Get the list of scan and mask filenames
scan_files = os.listdir(scan_dir)
mask_files = os.listdir(mask_dir)

# Ensure the filenames match between scans and masks
assert len(scan_files) == len(mask_files), "Number of scans and masks do not match!"
for scan_file, mask_file in zip(scan_files, mask_files):
    assert scan_file == mask_file, f"Scan file {scan_file} does not match mask file {mask_file}!"

# Split the filenames into train and test sets
train_files, test_files = train_test_split(scan_files, test_size=0.2, random_state=42)

### Preprocessing the data
Here I preprocess the dataset so that:
- The scans and the masks are **resized** to be the same shape, here I pick (350, 500) since it's the average shape
- The scans are made to be true **grayscale**
- The masks are used to extract the coordinates of the **groundtruth edge** (ELM)

In [ ]:
def preprocess_scan(scan_path):
    # Load the OCT scan image as grayscale
    scan = cv2.imread(scan_path, cv2.IMREAD_GRAYSCALE)
    # Resize the scan to have shape (350, 500)
    scan = cv2.resize(scan, (500, 350))
    # Normalise pixel intensities to the range [0, 1]
    scan = scan.astype(np.float32) / 255.0

    return scan


def extract_elm_coords(mask_path):
    # Load the mask image as grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    # Resize the mask to have shape (350, 500)
    mask = cv2.resize(mask, (500, 350))

    # Threshold the mask to create a binary image
    _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Find the non-zero pixel coordinates in the binary mask
    elm_coords = np.argwhere(binary_mask > 0)
    return elm_coords


def load_dataset(files, scan_directory, mask_directory):
    scans = []
    elm_coords_list = []
    for file in files:
        scan_path = os.path.join(scan_directory, file)
        mask_path = os.path.join(mask_directory, file)

        # Load and preprocess the OCT scan
        scan = preprocess_scan(scan_path)
        scans.append(scan)

        # Extract the ELM coordinates from the mask
        elm_coords = extract_elm_coords(mask_path)
        elm_coords_list.append(elm_coords)

    return scans, elm_coords_list


# Load the training data
train_scans, train_elm_coords = load_dataset(train_files, scan_dir, mask_dir)

# Load the testing data
test_scans, test_elm_coords = load_dataset(test_files, scan_dir, mask_dir)

# Print some information about the loaded data
print(f"Number of training samples: {len(train_scans)}")
print(f"Number of testing samples: {len(test_scans)}")
print(f"Shape of first training scan: {train_scans[0].shape}")  # Checking the resizing was done correctly
print(f"Number of ELM coordinates in first training mask: {len(train_elm_coords[0])}")

In [ ]:
def display_elm_on_scan(scan, elm_coords, figsize=(10, 10), compare=False, true_elm_coords=None):
    """
    Display the ELM coordinates overlaid on the OCT scan.
    
    Parameters:
    - scan: 2D numpy array representing the OCT scan
    - elm_coords: numpy array of shape (N, 2) representing the predicted ELM coordinates
    - figsize: tuple, size of the figure (width, height) in inches
    - compare: boolean, if True, compare predicted ELM coordinates with true ELM coordinates
    - true_elm_coords: numpy array of shape (N, 2) representing the true ELM coordinates (required if compare is True)
    """

    plt.figure(figsize=figsize)

    # Display the scan
    plt.imshow(scan, cmap='gray')

    # Overlay the predicted ELM coordinates
    plt.scatter(elm_coords[:, 1], elm_coords[:, 0], c='red', s=1, alpha=0.5, label='Predicted ELM')

    if compare:
        if true_elm_coords is None:
            raise ValueError("true_elm_coords must be provided when compare is True")

        # Overlay the true ELM coordinates
        plt.scatter(true_elm_coords[:, 1], true_elm_coords[:, 0], c='green', s=1, alpha=0.5, label='True ELM')

        # Calculate the DICE coefficient
        dice_coef = gpet_utils.trace_dicecoef(elm_coords, true_elm_coords)
        plt.title(f'OCT Scan with ELM Coordinates (DICE: {dice_coef:.4f})')
    else:
        plt.title('OCT Scan with ELM Coordinates')

    plt.legend()
    plt.axis()
    plt.tight_layout()
    plt.show()

In [ ]:
def compute_edge_map(scan):
    # Create a kernel for edge detection
    kernel = gpet_utils.kernel_builder(size=(11, 5), unit=False, normalize=True)
    # Compute the gradient image
    grad_img = gpet_utils.comp_grad_img(scan, kernel)
    return grad_img

In [ ]:
# Test the function using a singular scan
test_idx = 0
test_scan = train_scans[test_idx]
test_elm = train_elm_coords[test_idx]
test_edge_map = compute_edge_map(test_scan)

In [ ]:
display_elm_on_scan(test_scan, test_elm)

In [ ]:
# Get initialization edges from the true_elm_coords aka mask
init = test_elm[[0, -1], :][:, [1, 0]]
# Switch rows so the x's are in ascending order
init[[0, 1]] = init[[1, 0]]
init

In [ ]:
# Display the edge map with ELM annotated
display_elm_on_scan(test_edge_map, test_elm)

# TODO: TRY TO DO COMBINE KERNELS TO GET BETTER RESULTS.
*MIGHT BE EXP.*


In [ ]:
gp_kwargs = dict(
    kernel_options={'kernel': 'Matern', 'sigma_f': 60, 'length_scale': 20, 'nu': 2.5},
    delta_x=8,
    score_thresh=0.5,
    N_samples=1000,
    seed=1,
    noise_y=0.5,
    keep_ratio=0.1,
    pixel_thresh=5,
    fix_endpoints=True,
    return_std=True
)

# Instantiate algorithm using parameters in __init__()
noisy_trace = gpet.GP_Edge_Tracing(init, test_edge_map, **gp_kwargs)

In [ ]:
# __call__() parameters and run algorithm on test image
print_final_diagnostics = False
show_init_post = True
show_post_iter = True
verbose = True
edge_pred, edge_credint = noisy_trace(print_final_diagnostics,
                                      show_init_post,
                                      show_post_iter,
                                      verbose)

In [ ]:
display_elm_on_scan(test_scan, edge_pred, compare=True, true_elm_coords=test_elm)

**Experimenting with the use of the variable `obs`**

In [ ]:
test_elm

**Here I randomly select 100 xy-points as the `obs` variable.**

In [ ]:
# Randomly select 100 rows
test_obs = test_elm[np.random.choice(test_elm.shape[0], size=100, replace=False), :]
test_obs = test_obs[test_obs[:, 0].argsort()]
test_obs

In [ ]:
display_elm_on_scan(test_scan, test_obs)

In [ ]:
gp_kwargs_obs = dict(
    kernel_options={'kernel': 'Matern', 'sigma_f': 60, 'length_scale': 20, 'nu': 2.5},
    delta_x=8,
    score_thresh=0.5,
    N_samples=1000,
    seed=1,
    noise_y=0.5,
    keep_ratio=0.1,
    pixel_thresh=5,
    fix_endpoints=True,
    return_std=True,
    obs=test_obs
)

# Instantiate algorithm using parameters in __init__()
noisy_trace_w_obs = gpet.GP_Edge_Tracing(init, test_edge_map, **gp_kwargs_obs)

In [ ]:
# __call__() parameters and run algorithm on test image
print_final_diagnostics = False
show_init_post = True
show_post_iter = True
verbose = True
edge_pred_obs, edge_credint_obs = noisy_trace_w_obs(print_final_diagnostics,
                                                    show_init_post,
                                                    show_post_iter,
                                                    verbose)

In [ ]:
display_elm_on_scan(test_scan, edge_pred_obs, compare=True, true_elm_coords=test_elm)

**In reality, we will not have the luxury of having the masks for the ELM annotations hence this is impractical, but I will implement the Bayesian Recursive Scheme where the `obs` variable will be dependent on the prior scans and their predictions, or a subset will be predetermined to come up with knowledge.**

**Now, we will try to improve the accuracy of the model by selecting linearly spaced xy-points from `test_elm_coords`.**

In [ ]:
num_rows = test_elm.shape[0]

start, stop = 0, num_rows - 1
row_indices = np.linspace(start, stop, num=100, dtype=int)

test_obs_lp = test_elm[row_indices, :]

test_obs_lp = test_obs_lp[test_obs_lp[:, 0].argsort()]

test_obs_lp

In [ ]:
display_elm_on_scan(test_scan, test_obs_lp)

In [ ]:
gp_kwargs_obs_lp = dict(
    kernel_options={'kernel': 'Matern', 'sigma_f': 60, 'length_scale': 20, 'nu': 2.5},
    delta_x=8,
    score_thresh=0.5,
    N_samples=1000,
    seed=1,
    noise_y=0.5,
    keep_ratio=0.1,
    pixel_thresh=5,
    fix_endpoints=True,
    return_std=True,
    obs=test_obs_lp
)

# Instantiate algorithm using parameters in __init__()
noisy_trace_w_obs_lp = gpet.GP_Edge_Tracing(init, test_edge_map, **gp_kwargs_obs_lp)

In [ ]:
# __call__() parameters and run algorithm on test image
print_final_diagnostics = False
show_init_post = True
show_post_iter = True
verbose = True
edge_pred_lp, edge_credint_lp = noisy_trace(print_final_diagnostics,
                                            show_init_post,
                                            show_post_iter,
                                            verbose)

In [ ]:
display_elm_on_scan(test_scan, edge_pred_lp, compare=True, true_elm_coords=test_elm)